In [ ]:

from sqlalchemy import create_engine
import pandas as pd
from os import environ
import numpy as np
engine = create_engine('postgresql://postgres:%s@localhost:5432/postgres'%str("huggi678213"))

In [ ]:
df = pd.read_sql_query('select * from "asx_data"',con=engine)
#df = df.set_index(["symbol","timestamp"])

In [ ]:
df.head()

In [ ]:
df["symbol"].value_counts()

In [ ]:
# create one fucking huge df
df = df.set_index(["symbol","timestamp"]).reset_index()
df[df["symbol"]=="APT"].head()

In [ ]:

print(df.shape)
df.head()

In [ ]:
# change columns
columns = ["price","quantity","volume","noOrders","turnover-priceTimesQuantity","algorithm1"]
for col in columns:
    df["%sChangePct" % col] = df[col].pct_change()
# signal
df["signal"] = np.sign(df["priceChangePct"])
    
df = df.drop(["symbol","timestamp"],axis=1)
print(df.shape)
df[columns].head()

In [ ]:
from sklearn.model_selection import train_test_split

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0.)

Y = df["signal"].shift(-1)
# cheat a little and set all 0 (no change) to buy
Y = Y.fillna(0.)
Y = Y.replace(0.,1.)

x_train, x_test, y_train, y_test = train_test_split(df, Y, test_size=0.23, shuffle=False)

In [ ]:
# min max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
from sklearn.linear_model import LassoCV

clf = LassoCV()
clf.fit(x_train,y_train)
scr = clf.score(x_test,y_test)
print(scr)

# lstm

In [ ]:
def threeD(x):
    x_train = []
    y_train = []
    window = 60
    target = -1 # -1 should be ma5_win, change indicator bla bla 
    for i in range(window, len(x)):
        x_train.append(x[i-window:i])
        y_train.append(x[i, target]) 
    x_train, y_train = np.array(x_train), np.array(y_train)
    return x_train,y_train

x_train,y_train = threeD(x_train)
x_test,y_test = threeD(x_test)

# next shuffle that shit
np.random.shuffle(x_train)
np.random.shuffle(x_test)

print(x_train.shape,x_test.shape)

In [ ]:
import pyarrow as pa
np.savez_compressed("data/asxdata",x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint


redlr = ReduceLROnPlateau(monitor="val_loss",patience=100)
es = EarlyStopping(monitor="val_loss",patience=200)
mcp_save = ModelCheckpoint('data/lstm_stock.hdf5', save_best_only=True, monitor='val_loss', mode='min')



classifier = Sequential()

classifier.add(LSTM(units = 128, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 256, return_sequences = True))
classifier.add(Dropout(0.2))

# classifier.add(LSTM(units = 50, return_sequences = True))
# classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 64))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 1))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy']) # y_train[y_train==0.5] = 0


In [ ]:
classifier.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=200, batch_size=64,callbacks=[redlr,es,mcp_save] )